In [1]:
pip install openai langchain python-dotenv


In [2]:
!pip install transformers



In [10]:
# Install transformers if not installed
!pip install transformers --quiet

import torch
from transformers import BlenderbotTokenizer, BlenderbotForConditionalGeneration

# Force CPU device
device = torch.device("cpu")

# Model & tokenizer loading
model_name = "facebook/blenderbot-400M-distill"
tokenizer = BlenderbotTokenizer.from_pretrained(model_name)
model = BlenderbotForConditionalGeneration.from_pretrained(model_name).to(device)

print("Model and tokenizer loaded successfully on CPU!")


Model and tokenizer loaded successfully on CPU!


In [22]:
import gradio as gr

# -----------------------------
# 1. FAQ knowledge base
# -----------------------------
FAQ_KB = {
    "return policy": "Our return policy allows you to return any product within 7 days of delivery, provided it is in original condition with packaging.",
    "track order": "You can track your order anytime by logging into your account and visiting the 'Orders' section.",
    "payment methods": "We accept Visa, MasterCard, American Express, PayPal, and Google Pay for your convenience.",
    "warranty": "All our products come with a 1-year manufacturer warranty covering hardware defects.",
    "specifications": "Please specify which product you'd like to know the specifications about — for example, our latest smartphone or laptop."
}

FAQ_KEYWORDS = {
    "return policy": ["return", "policy", "refund", "return policy", "return policies"],
    "track order": ["track order", "where is my order", "order status", "tracking"],
    "payment methods": ["payment methods", "how to pay", "payment options", "payment"],
    "warranty": ["warranty", "guarantee", "manufacturer warranty"],
    "specifications": ["specifications", "specs", "details", "product specs"]
}

# -----------------------------
# 2. Chatbot system prompt
# -----------------------------
system_prompt = (
    "You are a helpful and professional customer support assistant for an electronics company. "
    "Your company sells smartphones, laptops, and accessories. "
    "Always answer politely and concisely with relevant company policies and product information. "
    "Do not say you are a customer service representative. "
    "If you do not know the answer, politely say you will find out and get back to the customer."
)

# -----------------------------
# 3. BlenderBot-based response generator
# -----------------------------
chat_history = []

def generate_blenderbot_response(user_message):
    context_lines = chat_history[-4:]
    prompt = system_prompt + "\n" + "\n".join(context_lines) + f"\nCustomer: {user_message}\nSupport:"

    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        padding=True,       # Pad to longest in batch (only one input here)
        max_length=256
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}

    try:
        output_ids = model.generate(
            **inputs,
            max_length=60,
            pad_token_id=tokenizer.pad_token_id,   # Use tokenizer’s pad_token_id here
            do_sample=True,
            top_p=0.9,
            temperature=0.7
        )
        decoded = tokenizer.decode(output_ids[0], skip_special_tokens=True)

        response = decoded.replace(prompt, "").strip()
        if not response:
            response = "Hmm, can you clarify that?"
        return response

    except Exception as e:
        import traceback
        print("----- Detailed error trace -----")
        traceback.print_exc()
        print("----- End error trace -----")
        return "Sorry, I encountered an internal error. Please try again."



# -----------------------------
# 4. Main chatbot function
# -----------------------------
def chat(user_input, history):
    user_input_lower = user_input.lower()
    for key, keywords in FAQ_KEYWORDS.items():
        if any(keyword in user_input_lower for keyword in keywords):
            answer = FAQ_KB[key]
            chat_history.append(f"Customer: {user_input}")
            chat_history.append(f"Support: {answer}")
            return answer

    # Else, generate response
    chat_history.append(f"Customer: {user_input}")
    response = generate_blenderbot_response(user_input)
    chat_history.append(f"Support: {response}")
    return response

# -----------------------------
# 5. Gradio UI
# -----------------------------
gr.ChatInterface(
    fn=chat,
    title="ElectroAssist – Support Chatbot",
    description="Ask anything about your electronic products, orders, payments, or warranty.",
    theme="default",
).launch(share=True)


/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:339: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4412635812a1e7cf08.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
